In [1]:
%%pyspark
blob_account_name = "vialfillstore"
blob_container_name = "vialfilldata"
from pyspark.sql import SparkSession

sc = SparkSession.builder.getOrCreate()
token_library = sc._jvm.com.microsoft.azure.synapse.tokenlibrary.TokenLibrary
blob_sas_token = token_library.getConnectionString("vialstore")

spark.conf.set(
    'fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name),
    blob_sas_token)
df = spark.read.load('wasbs://vialfilldata@vialfillstore.blob.core.windows.net/ftedge_gateway_01/2022/10/20/0_f90ab0ed97844fa781de699ccf0dfd7f_1.json', format='json')
#display(df.limit(10))

In [2]:
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

In [3]:
from pyspark.sql.types import *

schema = StructType([
  StructField("ConnectionDeviceId", StringType()),
  StructField("EnqueuedTime", StringType()),
  StructField("EventEnqueuedUtcTime", StringType()),
  StructField("EventProcessedUtcTime", StringType()),
  StructField("gatewayData", ArrayType(
      StructType([
          StructField("mimeType", StringType()),
          StructField("model_id", StringType()),
          StructField("tag_id", StringType()),
          StructField("vqts", ArrayType(
              StructType([
                      StructField('q', StringType()),
                      StructField('t', StringType()),
                      StructField('v', StringType())
                  ])
                )
            )
        ])
    )
)
])

In [4]:
jsonpath = "wasbs://vialfilldata@vialfillstore.blob.core.windows.net/ftedge_gateway_01/2022/10/20/*.json"

In [5]:
#df = spark.read.schema(schema).json(jsonpath)

In [5]:
df = spark.read.json(jsonpath)

In [61]:
display(df.limit(1))

In [6]:
from pyspark.sql import functions as f

df2 = df.withColumn("gatewayDataexp",f.explode('gatewayData'))
df3 = df2.withColumn("vqtsexp",f.explode('gatewayDataexp.vqts'))

In [7]:
display(df3.limit(10))

In [8]:
df_final = df3.selectExpr("ConnectionDeviceId","EventProcessedUtcTime","EventEnqueuedUtcTime","ConnectionDeviceId","gatewayDataexp.model_id as model_id","gatewayDataexp.tag_id as tag_id"
,"vqtsexp.v as v","vqtsexp.q as q","vqtsexp.t as t"
)

In [9]:
display(df_final.limit(100))

In [ ]:
#df2.select("ConnectionDeviceId", "tag_id", "vqts1.q", "vqts1.t", "vqts1.v").repartition(1).write.mode("overwrite").parquet('abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/rawdata/')

In [10]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

#pivotDF = df1.groupBy('t', 'tag_id').pivot("tag_id").agg(F.collect_list("v"))


In [12]:
from pyspark.sql.functions import *

df_final = df_final.withColumn('tagidformatted', regexp_replace('tag_id', 'ra-cip-value://driver-cip/192.168.1.11/Production_History', ''))

In [15]:
#display(df1.limit(5))

In [16]:
#display(df1.where(df1.tagidformatted == 'DryFillFinishTimeYear'))

In [13]:
df_final = df_final.withColumn('tagidformatted', regexp_replace('tagidformatted', "[^0-9A-Za-z]", ''))
df_final = df_final.withColumn('tagidformatted', regexp_replace('tagidformatted', "0", ''))
#df1 = df1.withColumn('tagidformatted', regexp_replace('tagidformatted', "[.", ''))

In [14]:
display(df_final.limit(5))

In [21]:
#display(df1.groupBy('tagidformatted', 't').count())

In [ ]:
#display(pivotDF.where(pivotDF[11] != '0'))

In [15]:
display(df_final.where(df_final.tagidformatted == 'DryFillFinishTimeYear'))

In [16]:
display(df_final.where((df_final.v == '2022') & (df_final.tagidformatted == 'CappingStationFinishTimeSecond')))

In [17]:
#display(df1.limit(10))

In [ ]:
#display(df1.where(df1.tag_id == 'ra-cip-value://driver-cip/192.168.1.11/Production_History[0].Dry_Fill_Finish_Time.Year'))

In [19]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

#pivotDF = df1.groupBy('t').pivot("tagidformatted").agg(F.collect_list("v"))
#pivotDF = df1.groupBy('t').pivot("tagidformatted").agg(F.last("v"))
#pivotDF = df1.groupBy('t').pivot("tagidformatted").agg(F.first("v"))
#pivotDF = df1.groupBy('t').pivot("tagidformatted").agg(F.avg("v"))
pivotDF = df_final.groupBy('t', 'ConnectionDeviceId','tagidformatted').pivot("tagidformatted").agg(F.first('v')).fillna(0)

In [20]:
display(pivotDF.limit(10))

In [ ]:
pivotDF.dtypes

In [21]:
display(pivotDF.where(pivotDF.DryFillFinishTimeYear != '0'))

In [23]:
pivotDF.repartition(1).write.mode("overwrite").parquet('abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/pivotdata/')

In [ ]:
pivotDF.columns

In [ ]:
display(pivotDF)

In [26]:
%%pyspark
df = spark.read.load('abfss://csiwkproot@csiwkpstore.dfs.core.windows.net/pivotdata/*.snappy.parquet', format='parquet')
display(df.limit(10))

In [27]:
display(df.where(df.DryFillFinishTimeYear != '0'))